## DIVIDIR UN VOLÚMEN DE GRUPO

Lo que quiere decir es separar un volumen de grupo, lo que quiere decir que se va a separar un volumen de grupo y crear uno nuevo.

Debemos estar en la máquina virtual con los 5 discos sin formato (si sus discos duros tienen formato, favor restaurar la instantánea apropiada)

Logueados como el usuario **root**, nos aseguramos de que tengamos los discos sin formato

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
```
***

Ahora revisamos el script para asegurarse que se formatearan los 5 discos

`cat 123`


***
```
root@debian12lvm1:~# cat 123
#!/bin/bash
#Estos comandos Establecen que van a ser discos msdos master boot record 
parted /dev/sdb mklabel msdos
parted /dev/sdc mklabel msdos
parted /dev/sdd mklabel msdos
parted /dev/sde mklabel msdos
parted /dev/sdf mklabel msdos

#Estos comandos crean una particion primaria y le hacen un set a lvm
parted -s /dev/sdb unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdc unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdd unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sde unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdf unit mib mkpart primary 1 100% set 1 lvm on

# Solo si es RHEL8
#Este comando le dice al kernel que reconozca esto nuevo
#udevadm settle

# De lo contrario ejecute
partprobe -s

```
***


El procedimiento para crear el RAID0 comienza con ejecutar el script 123 para los 5 discos duros

`sh 123`

***
```
root@debian12lvm1:~# sh 123
Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

/dev/sda: msdos partitions 1 2 <5>                                        
/dev/sdb: msdos partitions 1
/dev/sdc: msdos partitions 1
/dev/sdd: msdos partitions 1
/dev/sde: msdos partitions 1
/dev/sdf: msdos partitions 1
```
***

Ahora ya tenemos nuestro espacio listo para comenzar a trabajar, y para comenzar este ejercicio vamos a crear un nuevo grupo

`vgcreate new1 /dev/sdb1 /dev/sdc1`

***
```
root@debian12lvm1:~# vgcreate new1 /dev/sdb1 /dev/sdc1
  Physical volume "/dev/sdb1" successfully created.
  Physical volume "/dev/sdc1" successfully created.
  Volume group "new1" successfully created
```
***

Ahora crearemos un volúmen lógico

`lvcreate -n lv1 new1 -l +100%FREE`

***
```
root@debian12lvm1:~# lvcreate -n lv1 new1 -l +100%FREE
  Logical volume "lv1" created.
```
***

Ahora creamos el sistema de archivos

`mkfs.ext4 /dev/new1/lv1`

***
```
mkfs.ext4 /dev/new1/lv1 
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 2619392 4k blocks and 655360 inodes
Filesystem UUID: 299d5a7a-3323-4611-942f-170b132ea0d2
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done
```
***

Ahora lo montamos

`mount /dev/new1/lv1 /mnt/`

Y ahora usaremos el comando siguiente para ver los volumenes fisicos y sus grupos de volumenes asociados

`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdb1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / 0    free]
  Total: 2 [9.99 GiB] / in use: 2 [9.99 GiB] / in no VG: 0 [0   ]
```
***

El grupo de volumenes esta usando al 100% cada unos de los volúmenes físicos y no hay espacio disponible, para agregar mas espacio al volumen lo vamos a extender ejecutando el siguiente comando

`vgextend new1 /dev/sdd1`

***
```
root@debian12lvm1:~# vgextend new1 /dev/sdd1
  Physical volume "/dev/sdd1" successfully created.
  Volume group "new1" successfully extended
```
***

Si ahora ejecutamos `pvscan` se puede observar que se tienen 5 GB libres

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdb1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdd1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  Total: 3 [<14.99 GiB] / in use: 3 [<14.99 GiB] / in no VG: 0 [0   ]
```
***

Ahora podemos crear un archivo de 3GB con datos aleatorios usando el siguiente comando que tarda un poco

`dd if=/dev/urandom of=/mnt/1.txt bs=3M count=1024`

***
```
dd if=/dev/urandom of=/mnt/1.txt bs=3M count=1024
1024+0 records in
1024+0 records out
3221225472 bytes (3.2 GB, 3.0 GiB) copied, 44.9934 s, 71.6 MB/s
```
***

Corroboramos que el archivo haya sido creado con el comando 

`ls -lash /mnt`

***
```
4.0K drwxr-xr-x  3 root root 4.0K feb 12 23:00 .
4.0K drwxr-xr-x 18 root root 4.0K ene 24 16:34 ..
3.1G -rw-r--r--  1 root root 3.0G feb 12 23:01 1.txt
 16K drwx------  2 root root  16K feb 12 22:50 lost+found
```
***

Ya tenemos datos en el volumen lógico distribuido en los discos, ahora ejecutamos nuevamente 

`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdb1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdd1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  Total: 3 [<14.99 GiB] / in use: 3 [<14.99 GiB] / in no VG: 0 [0   ]
```
***

#### ¿Cómo nosotros sacamos un disco físico de aqui?

Primero agreguemos mas discos al vg

`vgextend new1 /dev/sde1`

***
```
vgextend new1 /dev/sde1
  Physical volume "/dev/sde1" successfully created.
  Volume group "new1" successfully extended
```
***

`vgextend new1 /dev/sdf1`

***
```
vgextend new1 /dev/sdf1
  Physical volume "/dev/sdf1" successfully created.
  Volume group "new1" successfully extended
```
***

Ahora, si ejecutamos 

`pvscan`

se podrá ver que tenemos 3 volumenes de grupo que no están siendo utilizados

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdb1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdd1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sde1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sdf1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  Total: 5 [24.98 GiB] / in use: 5 [24.98 GiB] / in no VG: 0 [0   ]
```
***


Si ejecutamos

`vgs`

Veremos que tenemos libres casi 15 GB que son la suma de los tres volumenes libres

***
```
root@debian12lvm1:~# vgs
  VG   #PV #LV #SN Attr   VSize  VFree  
  new1   5   1   0 wz--n- 24.98g <14.99g
```
***

El primer método para separa un disco, es mover los extends que hay en el disco que nosotros queremos mover, podemos usar el comando `pvmove` con el volúmen fisico que queremos sacar un ejemplo del comando sería el siguiente, la opcion -i5 muestra el progreso cada 5 segundos

`pvmove -i5 /dev/sdb1`

***
```
root@debian12lvm1:~# pvmove -i5 /dev/sdb1
  /dev/sdb1: Moved: 0.31%
  /dev/sdb1: Moved: 6.25%
  /dev/sdb1: Moved: 9.85%
  /dev/sdb1: Moved: 12.74%
  /dev/sdb1: Moved: 15.72%
  /dev/sdb1: Moved: 18.76%
  /dev/sdb1: Moved: 21.58%
  /dev/sdb1: Moved: 24.47%
  /dev/sdb1: Moved: 27.44%
  /dev/sdb1: Moved: 30.34%
  /dev/sdb1: Moved: 33.46%
  /dev/sdb1: Moved: 37.61%
  /dev/sdb1: Moved: 44.25%
  /dev/sdb1: Moved: 48.79%
  /dev/sdb1: Moved: 53.48%
  /dev/sdb1: Moved: 57.94%
  /dev/sdb1: Moved: 62.47%
  /dev/sdb1: Moved: 72.17%
  /dev/sdb1: Moved: 97.89%

```
***

¿Y para donde movió los datos?

Los datos pudieron ser movidos para alguno de los discos libres que agregamos pero por lo general lo mueve para el siguiente disco disponible **/dev/sdd1** (ver la salida del comando pvscan).

`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdb1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdd1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sde1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sdf1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  Total: 5 [24.98 GiB] / in use: 5 [24.98 GiB] / in no VG: 0 [0   ]

```
***



Y ahora si podemos dividir el grupo de volumenes mediante el siguiente comando

`vgsplit new1 new2 /dev/sdb1`

***
```
root@debian12lvm1:~# vgsplit new1 new2 /dev/sdb1
  New volume group "new2" successfully split from "new1"
```
***

El dispositivo **/dev/sdb1** ahora pertenece al grupo **new2**

Si ejecutamos el siguiente comando podremos ver los dos grupos

`vgs`

***
```
root@debian12lvm1:~# vgs
  VG   #PV #LV #SN Attr   VSize  VFree 
  new1   4   1   0 wz--n- 19.98g  9.99g
  new2   1   0   0 wz--n- <5.00g <5.00g
```
***

Ahora podemos usar un nuevo comando para ver detalles del grupo

`vgdisplay new2`

***
```
root@debian12lvm1:~# vgdisplay new2 
  --- Volume group ---
  VG Name               new2
  System ID             
  Format                lvm2
  Metadata Areas        1
  Metadata Sequence No  2
  VG Access             read/write
  VG Status             resizable
  MAX LV                0
  Cur LV                0
  Open LV               0
  Max PV                0
  Cur PV                1
  Act PV                1
  VG Size               <5.00 GiB
  PE Size               4.00 MiB
  Total PE              1279
  Alloc PE / Size       0 / 0   
  Free  PE / Size       1279 / <5.00 GiB
  VG UUID               mWZEUU-eqiA-fdD4-sN6h-Uhj5-BEaA-bFufrB

```
***

Si ejecutamos el siguiente comando podremos ver como queda la asignación de los volumenes físicos

`pvs`

***
```
root@debian12lvm1:~# pvs
  PV         VG   Fmt  Attr PSize  PFree 
  /dev/sdb1  new2 lvm2 a--  <5.00g <5.00g
  /dev/sdc1  new1 lvm2 a--  <5.00g     0 
  /dev/sdd1  new1 lvm2 a--  <5.00g     0 
  /dev/sde1  new1 lvm2 a--  <5.00g <5.00g
  /dev/sdf1  new1 lvm2 a--  <5.00g <5.00g
```
***

Antes de dividir el volumen, algunas personas lo que hacen es desactivar el volumen y despues hacen el **vgsplit**, usan el siguiente comando

`lvchange -a n`

Ahora, por ejemplo, si queremos sacar los datos de **/dev/sdc1** y llevarlo específicamente a **/dev/sdf1** 

¿Cómo le decimos al sistema que haga eso?

los hacemos con el comando en donde la opcion **-b** lo ejecuta en background

`pvmove -b /dev/sdc1 /dev/sdf1`

y si ejecutamos 

`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdc1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sdd1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sde1   VG new1            lvm2 [<5.00 GiB / <5.00 GiB free]
  PV /dev/sdf1   VG new1            lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdb1   VG new2            lvm2 [<5.00 GiB / <5.00 GiB free]
```
***

Pero como se ejecuta en el background hay que darle tiempo para que mueva todos los datos

Una vez que mueve todos los datos, podemos hacer un split

`vgsplit new1 new3 /dev/sdc1`

***
```
root@debian12lvm1:~# vgsplit new1 new3 /dev/sdc1 
  New volume group "new3" successfully split from "new1"
```
***
